In [54]:
"""
ATFM
@author: xxx
"""
# Make sure this libraries are installed in your virtual environment
#from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
#from coopr.pyomo import *

# DEFINICIÓN DE LOS INPUTS

In [55]:
# Elements of the problem

#Dimensions
n_airports = 3
n_sectors = 4
n_flights = 12
time_intervals = 40

#Sets
T = list(range(time_intervals+1))
F = ['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12']
K = ['STAR 1','STAR 2','STAR 3', 'STAR 4', 'FAP'] 


# Departure capacity of airport K at time t
D_k = {}
def D_k_ini(D_k, time_intervals):
    for i in range(1, time_intervals+1):
        D_k["STAR 1", i] = 1
        D_k["STAR 2", i] = 1
        D_k["STAR 3", i] = 1
        D_k["STAR 4", i] = 1
        D_k["FAP", i]    = 1
    return D_k

# Arrival capacity of airport K at time t
A_k = {}
def A_k_ini(A_k, time_intervals):
    for i in range(1, time_intervals+1):
        A_k["STAR 1", i] = 1
        A_k["STAR 2", i] = 1
        A_k["STAR 3", i] = 1
        A_k["STAR 4", i] = 1
        A_k["FAP", i]    = 1
    return A_k

# Scheduled departure times
d_f = {F[0]: 1, F[1]: 2, F[2]: 3, F[3]: 1, F[4]: 2, F[5]: 3, F[6]: 1, F[7]: 2, F[8]: 3, F[9]: 1, F[10]: 2, F[11]: 3}

# Scheduled arrivals times
a_f = {F[0]: 1+9, F[1]: 2+9, F[2]: 3+9, F[3]: 1+9, F[4]: 2+9, F[5]: 3+9, F[6]: 1+9, F[7]: 2+9, F[8]: 3+9, F[9]: 1+9, F[10]: 2+9, F[11]: 3+9}


# Origin and Destination Airport
origin_f = {}
origin_f[F[0]]  = K[0]
origin_f[F[1]]  = K[0]
origin_f[F[2]]  = K[0]

origin_f[F[3]]  = K[1]
origin_f[F[4]]  = K[1]
origin_f[F[5]]  = K[1]

origin_f[F[6]]  = K[2]
origin_f[F[7]]  = K[2]
origin_f[F[8]]  = K[2]

origin_f[F[9]]  = K[3]
origin_f[F[10]] = K[3]
origin_f[F[11]] = K[3]

destination_f = {}
destination_f[F[0]]  = K[4]
destination_f[F[1]]  = K[4]
destination_f[F[2]]  = K[4]
destination_f[F[3]]  = K[4]
destination_f[F[4]]  = K[4]
destination_f[F[5]]  = K[4]
destination_f[F[6]]  = K[4]
destination_f[F[7]]  = K[4]
destination_f[F[8]]  = K[4]
destination_f[F[9]]  = K[4]
destination_f[F[10]] = K[4]
destination_f[F[11]] = K[4]

# Nodes for flight f
def N_f_ini(N_f):
    N_f["f1"]  = ("STAR 1","IAF 1","HOLD 1","FAP")
    N_f["f2"]  = ("STAR 1","IAF 1","HOLD 1","FAP")
    N_f["f3"]  = ("STAR 1","IAF 1","HOLD 1","FAP")

    N_f["f4"]  = ("STAR 2","IAF 2","HOLD 2","FAP")
    N_f["f5"]  = ("STAR 2","IAF 2","HOLD 2","FAP")
    N_f["f6"]  = ("STAR 2","IAF 2","HOLD 2","FAP")

    N_f["f7"]  = ("STAR 3","IAF 3","HOLD 3","FAP")
    N_f["f8"]  = ("STAR 3","IAF 3","HOLD 3","FAP")
    N_f["f9"]  = ("STAR 3","IAF 3","HOLD 3","FAP")

    N_f["f10"] = ("STAR 4","IAF 4","HOLD 4","FAP")
    N_f["f11"] = ("STAR 4","IAF 4","HOLD 4","FAP")
    N_f["f12"] = ("STAR 4","IAF 4","HOLD 4","FAP")
    return N_f
N_f = {} # esta linea crea el diccionario vacío

# Arcs for flight f
def arcs_ini(arcs):
    arcs["f1"]  = ("(STAR 1,IAF 1)","(IAF 1,HOLD 1)","(HOLD 1,FAP)")
    arcs["f2"]  = ("(STAR 1,IAF 1)","(IAF 1,HOLD 1)","(HOLD 1,FAP)")
    arcs["f3"]  = ("(STAR 1,IAF 1)","(IAF 1,HOLD 1)","(HOLD 1,FAP)")

    arcs["f4"]  = ("(STAR 2,IAF 2)","(IAF 2,HOLD 2)","(HOLD 2,FAP)")
    arcs["f5"]  = ("(STAR 2,IAF 2)","(IAF 2,HOLD 2)","(HOLD 2,FAP)")
    arcs["f6"]  = ("(STAR 2,IAF 2)","(IAF 2,HOLD 2)","(HOLD 2,FAP)")

    arcs["f7"]  = ("(STAR 3,IAF 3)","(IAF 3,HOLD 3)","(HOLD 3,FAP)")
    arcs["f8"]  = ("(STAR 3,IAF 3)","(IAF 3,HOLD 3)","(HOLD 3,FAP)")
    arcs["f9"]  = ("(STAR 3,IAF 3)","(IAF 3,HOLD 3)","(HOLD 3,FAP)")

    arcs["f10"] = ("(STAR 4,IAF 4)","(IAF 4,HOLD 4)","(HOLD 4,FAP)")
    arcs["f11"] = ("(STAR 4,IAF 4)","(IAF 4,HOLD 4)","(HOLD 4,FAP)")
    arcs["f12"] = ("(STAR 4,IAF 4)","(IAF 4,HOLD 4)","(HOLD 4,FAP)")
    return arcs
arcs = {}
#arcs_ini(arcs)

# Number of time units that flight f must spend in arc m,n
def l_fnm_ini(l_fnm):
    l_fnm["f1","(STAR 1,IAF 1)"] = 4
    l_fnm["f1","(IAF 1,HOLD 1)"] = 0
    l_fnm["f1","(HOLD 1,FAP)"]   = 5

    l_fnm["f2","(STAR 1,IAF 1)"] = 4
    l_fnm["f2","(IAF 1,HOLD 1)"] = 0
    l_fnm["f2","(HOLD 1,FAP)"]   = 5

    l_fnm["f3","(STAR 1,IAF 1)"] = 4
    l_fnm["f3","(IAF 1,HOLD 1)"] = 0
    l_fnm["f3","(HOLD 1,FAP)"]   = 5

    l_fnm["f4","(STAR 2,IAF 2)"] = 4
    l_fnm["f4","(IAF 2,HOLD 2)"] = 0
    l_fnm["f4","(HOLD 2,FAP)"]   = 5

    l_fnm["f5","(STAR 2,IAF 2)"] = 4
    l_fnm["f5","(IAF 2,HOLD 2)"] = 0
    l_fnm["f5","(HOLD 2,FAP)"]   = 5

    l_fnm["f6","(STAR 2,IAF 2)"] = 4
    l_fnm["f6","(IAF 2,HOLD 2)"] = 0
    l_fnm["f6","(HOLD 2,FAP)"]   = 5

    l_fnm["f7","(STAR 3,IAF 3)"] = 4
    l_fnm["f7","(IAF 3,HOLD 3)"] = 0
    l_fnm["f7","(HOLD 3,FAP)"]   = 5

    l_fnm["f8","(STAR 3,IAF 3)"] = 4
    l_fnm["f8","(IAF 3,HOLD 3)"] = 0
    l_fnm["f8","(HOLD 3,FAP)"]   = 5

    l_fnm["f9","(STAR 3,IAF 3)"] = 4
    l_fnm["f9","(IAF 3,HOLD 3)"] = 0
    l_fnm["f9","(HOLD 3,FAP)"]   = 5

    l_fnm["f10","(STAR 4,IAF 4)"] = 5
    l_fnm["f10","(IAF 4,HOLD 4)"] = 0
    l_fnm["f10","(HOLD 4,FAP)"]   = 5

    l_fnm["f11","(STAR 4,IAF 4)"] = 5
    l_fnm["f11","(IAF 4,HOLD 4)"] = 0
    l_fnm["f11","(HOLD 4,FAP)"]   = 5

    l_fnm["f12","(STAR 4,IAF 4)"] = 5
    l_fnm["f12","(IAF 4,HOLD 4)"] = 0
    l_fnm["f12","(HOLD 4,FAP)"]   = 5
    return l_fnm

l_fnm = {}
#l_fnm_ini(l_fnm)

# Scheduled time of arrival of flight f to arrive at node n (not considering possible delays)
def r_fn_ini(r_fn,model):
    for k in model.F:
        for i1,val1 in enumerate(model.N_f[k]):
            r_fn[k, val1] = model.d_f[k]
            for i2, val2 in enumerate(model.arcs[k]):
                if i2<i1:
                    r_fn[k, val1] = r_fn[k, val1]+model.l_fnm[k,val2]
    return r_fn
r_fn = {}

# maximum ground unit delays for flight f
# The ground delay in this case would be equal to the STAR delay as we are considering STARs as departures
def G_ini(G):
    G["f1"]  = 4
    G["f2"]  = 4
    G["f3"]  = 4
    G["f4"]  = 4
    G["f5"]  = 4
    G["f6"]  = 4
    G["f7"]  = 4
    G["f8"]  = 4
    G["f9"]  = 4
    G["f10"] = 4
    G["f11"] = 4
    G["f12"] = 4
    return G
G = {}
#G_ini(G)

# maximum airborne unit delays for flight f
# The airborne delay in this case would be equal to the holding delay
def A_ini(A):
    A["f1"]  = 5
    A["f2"]  = 5
    A["f3"]  = 5
    A["f4"]  = 5
    A["f5"]  = 5
    A["f6"]  = 5
    A["f7"]  = 5
    A["f8"]  = 5
    A["f9"]  = 5
    A["f10"] = 5
    A["f11"] = 5
    A["f12"] = 5
    return A
A = {}
#A_ini(A)


# Smallest time interval for flight f to arrive at node n
def T_fn_ini(T_fn,model):
    for k in model.F:
        for i,val in enumerate(model.N_f[k]):
            if i<2:
                T_fn[k,val] = np.linspace(model.r_fn[k,val], min(model.r_fn[k,val]+model.G[k],time_intervals) , min(model.r_fn[k,val]+model.G[k],time_intervals)-model.r_fn[k,val]+1)
            if i>=2:
                T_fn[k, val] = np.linspace(model.r_fn[k,val], min(model.r_fn[k,val]+model.G[k]+model.A[k],time_intervals) , min(model.r_fn[k,val]+model.G[k]+model.A[k],time_intervals)-model.r_fn[k,val]+1)
    return T_fn
T_fn = {}

# Unit cost for air delay
def ca_ini(ca):
    ca["f1"]  = 2.5
    ca["f2"]  = 2.5
    ca["f3"]  = 2.5
    ca["f4"]  = 2.5
    ca["f5"]  = 2.5
    ca["f6"]  = 2.5
    ca["f7"]  = 2.5
    ca["f8"]  = 2.5
    ca["f9"]  = 2.5
    ca["f10"] = 2.5
    ca["f11"] = 2.5
    ca["f12"] = 2.5
    return ca
ca = {}
ca_ini(ca)

# Unit cost for ground delay 
def cg_ini(cg):
    cg["f1"]  = 1.5
    cg["f2"]  = 1.5
    cg["f3"]  = 1.5
    cg["f4"]  = 1.5
    cg["f5"]  = 1.5
    cg["f6"]  = 1.5
    cg["f7"]  = 1.5
    cg["f8"]  = 1.5
    cg["f9"]  = 1.5
    cg["f10"] = 1.5
    cg["f11"] = 1.5
    cg["f12"] = 1.5
    return cg
cg = {}
cg_ini(cg)

print('origin os the aproximation:', origin_f)
print('Times', T)
print('Set of Times to overfly node n for flight f', T_fn)
print('Nf', N_f)

origin os the aproximation: {'f1': 'STAR 1', 'f2': 'STAR 1', 'f3': 'STAR 1', 'f4': 'STAR 2', 'f5': 'STAR 2', 'f6': 'STAR 2', 'f7': 'STAR 3', 'f8': 'STAR 3', 'f9': 'STAR 3', 'f10': 'STAR 4', 'f11': 'STAR 4', 'f12': 'STAR 4'}
Times [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
Set of Times to overfly node n for flight f {}
Nf {}


# CREACIÓN DEL MODELO E INICIALIZACIÓN

In [56]:
############### Create a concrete model ###############
model = ConcreteModel('AMAN')

In [57]:
############### Initialize Sets and Parameters ###############
############### FEW examples are given below (complete until you get the print that you will find in the cell below) ###############

#
model.T = Set(initialize=T, ordered=True)
model.F = Set(initialize=F, ordered=True)
model.K = Set(initialize=K, ordered=True)

model.D_k = Param(model.K, model.T, initialize=D_k_ini({}, time_intervals), default=0)
model.A_k = Param(model.K, model.T, initialize=A_k_ini({}, time_intervals), default=0)

model.d_f = Param(model.F, initialize=d_f)
model.a_f = Param(model.F, initialize=a_f)

model.origin_f = Param(model.F, initialize=origin_f, within=Any)
model.destination_f = Param(model.F, initialize=destination_f, within=Any)

model.N_f = Set(N_f.keys(), initialize=N_f_ini(N_f), ordered=True)
model.arcs = Set(arcs.keys(), initialize=arcs_ini(arcs), ordered=True)

model.l_fnm = Param(l_fnm.keys(), initialize=l_fnm_ini(l_fnm))
model.r_fn = Param(r_fn.keys(), initialize=r_fn_ini(r_fn,model))

model.G = Param(model.F, initialize=G_ini({}))
model.A = Param(model.F, initialize=A_ini({}))

model.T_fn = Set(T_fn.keys(), initialize=T_fn_ini(T_fn, model)) 

model.ca = Param(ca.keys(), initialize=ca_ini(ca)) 
model.cg = Param(cg.keys(), initialize=cg_ini(cg)) 


print('T_fn:', T_fn)
print('N_f:', N_f)
print('arcs:', arcs)
print('r_fn:', r_fn)
print('l_fnm:', l_fnm)


T_fn: {('f1', 'STAR 1'): array([1., 2., 3., 4., 5.]), ('f1', 'IAF 1'): array([5., 6., 7., 8., 9.]), ('f1', 'HOLD 1'): array([ 5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14.]), ('f1', 'FAP'): array([10., 11., 12., 13., 14., 15., 16., 17., 18., 19.]), ('f2', 'STAR 1'): array([2., 3., 4., 5., 6.]), ('f2', 'IAF 1'): array([ 6.,  7.,  8.,  9., 10.]), ('f2', 'HOLD 1'): array([ 6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15.]), ('f2', 'FAP'): array([11., 12., 13., 14., 15., 16., 17., 18., 19., 20.]), ('f3', 'STAR 1'): array([3., 4., 5., 6., 7.]), ('f3', 'IAF 1'): array([ 7.,  8.,  9., 10., 11.]), ('f3', 'HOLD 1'): array([ 7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16.]), ('f3', 'FAP'): array([12., 13., 14., 15., 16., 17., 18., 19., 20., 21.]), ('f4', 'STAR 2'): array([1., 2., 3., 4., 5.]), ('f4', 'IAF 2'): array([5., 6., 7., 8., 9.]), ('f4', 'HOLD 2'): array([ 5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14.]), ('f4', 'FAP'): array([10., 11., 12., 13., 14., 15., 16., 17., 18., 19.]), ('f5', 

You should get this print
T_fn: {('f1', 'A'): array([1., 2., 3.]), ('f1', 'q(A)'): array([1., 2., 3.]), ('f1', 'a'): array([ 6.,  7.,  8.,  9., 10., 11., 12., 13.]), ('f1', 'b'): array([14., 15., 16., 17., 18., 19., 20., 21.]), ('f1', 'C'): array([15., 16., 17., 18., 19., 20., 21., 22.]), ('f2', 'A'): array([1., 2., 3., 4.]), ('f2', 'q(A)'): array([1., 2., 3., 4.]), ('f2', 'e'): array([ 6.,  7.,  8.,  9., 10., 11., 12., 13.]), ('f2', 'B'): array([13., 14., 15., 16., 17., 18., 19., 20.]), ('f3', 'B'): array([17., 18., 19., 20.]), ('f3', 'q(B)'): array([17., 18., 19., 20.]), ('f3', 'f'): array([20., 21., 22., 23., 24., 25., 26., 27.]), ('f3', 'C'): array([23., 24., 25., 26., 27., 28., 29., 30.])}

N_f: {'f1': ('A', 'q(A)', 'a', 'b', 'C'), 'f2': ('A', 'q(A)', 'e', 'B'), 'f3': ('B', 'q(B)', 'f', 'C')}

# DEFINICIÓN DE VARIABLES W

In [58]:
############### Define Variables ###############

var = {}

model.w = Var(
    [(f,n) 
     for f in model.F 
     for n in model.N_f[f]],
    model.T,
    within=Binary,
    initialize=0
)
# Estamos creando variables para cada combinación (f,n) EN TODOS LOS TIEMPOS (model.T), no en tiempos factibles (T_fn)
model.w.pprint()

w : Size=1968, Index={('f1', 'STAR 1'), ('f1', 'IAF 1'), ('f1', 'HOLD 1'), ('f1', 'FAP'), ('f2', 'STAR 1'), ('f2', 'IAF 1'), ('f2', 'HOLD 1'), ('f2', 'FAP'), ('f3', 'STAR 1'), ('f3', 'IAF 1'), ('f3', 'HOLD 1'), ('f3', 'FAP'), ('f4', 'STAR 2'), ('f4', 'IAF 2'), ('f4', 'HOLD 2'), ('f4', 'FAP'), ('f5', 'STAR 2'), ('f5', 'IAF 2'), ('f5', 'HOLD 2'), ('f5', 'FAP'), ('f6', 'STAR 2'), ('f6', 'IAF 2'), ('f6', 'HOLD 2'), ('f6', 'FAP'), ('f7', 'STAR 3'), ('f7', 'IAF 3'), ('f7', 'HOLD 3'), ('f7', 'FAP'), ('f8', 'STAR 3'), ('f8', 'IAF 3'), ('f8', 'HOLD 3'), ('f8', 'FAP'), ('f9', 'STAR 3'), ('f9', 'IAF 3'), ('f9', 'HOLD 3'), ('f9', 'FAP'), ('f10', 'STAR 4'), ('f10', 'IAF 4'), ('f10', 'HOLD 4'), ('f10', 'FAP'), ('f11', 'STAR 4'), ('f11', 'IAF 4'), ('f11', 'HOLD 4'), ('f11', 'FAP'), ('f12', 'STAR 4'), ('f12', 'IAF 4'), ('f12', 'HOLD 4'), ('f12', 'FAP')}*T
    Key                   : Lower : Value : Upper : Fixed : Stale : Domain
         ('f1', 'FAP', 0) :     0 :     0 :     1 : False : False : Binar

# DEFICIÓN DE LA FUNCIÓN OBJETIVO

In [59]:
### Define Objective Function ###
def objective_rule(model):
    objective_expr = 0
    for f in model.F:
        # --- COSTE EN AIRE ---
        n_dest = destination_f[f]
        for t in model.T_fn [f, n_dest]:
            prev_term = model.w[f, n_dest, t-1]
            objective_expr += ca[f] * (model.w[f, n_dest, t] - prev_term) * (t - model.a_f[f])

    return objective_expr

model.OBJ = Objective(rule=objective_rule, sense=minimize)
model.OBJ.pprint()


OBJ : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 2.5*(w[f1,FAP,10] - w[f1,FAP,9])*0.0 + 2.5*(w[f1,FAP,11] - w[f1,FAP,10]) + 2.5*(w[f1,FAP,12] - w[f1,FAP,11])*2.0 + 2.5*(w[f1,FAP,13] - w[f1,FAP,12])*3.0 + 2.5*(w[f1,FAP,14] - w[f1,FAP,13])*4.0 + 2.5*(w[f1,FAP,15] - w[f1,FAP,14])*5.0 + 2.5*(w[f1,FAP,16] - w[f1,FAP,15])*6.0 + 2.5*(w[f1,FAP,17] - w[f1,FAP,16])*7.0 + 2.5*(w[f1,FAP,18] - w[f1,FAP,17])*8.0 + 2.5*(w[f1,FAP,19] - w[f1,FAP,18])*9.0 + 2.5*(w[f2,FAP,11] - w[f2,FAP,10])*0.0 + 2.5*(w[f2,FAP,12] - w[f2,FAP,11]) + 2.5*(w[f2,FAP,13] - w[f2,FAP,12])*2.0 + 2.5*(w[f2,FAP,14] - w[f2,FAP,13])*3.0 + 2.5*(w[f2,FAP,15] - w[f2,FAP,14])*4.0 + 2.5*(w[f2,FAP,16] - w[f2,FAP,15])*5.0 + 2.5*(w[f2,FAP,17] - w[f2,FAP,16])*6.0 + 2.5*(w[f2,FAP,18] - w[f2,FAP,17])*7.0 + 2.5*(w[f2,FAP,19] - w[f2,FAP,18])*8.0 + 2.5*(w[f2,FAP,20] - w[f2,FAP,19])*9.0 + 2.5*(w[f3,FAP,12] - w[f3,FAP,11])*0.0 + 2.5*(w[f3,FAP,13] - w[f3,FAP,12]) + 2.5*(w[f3,FAP,14] - w[

# CAPACITY CONTRAINS

In [60]:
################### Define Departure Capacity constraint ####################

def departure_capacity_constraint(model,k,t):
    if t ==0:
        return Constraint.Skip #This is to skip the constratin at t=0.
    
    # Check if any flight has origin 'k'
    has_flights_from_k = any(model.origin_f[f] == k for f in model.F)
    if not has_flights_from_k:
        return Constraint.Skip 
    
    #Complete the rest of the instructions for the constraint.

    #flights_from_k crea una lista de los vuelos f que salen de cada aeropuerto
    flights_from_k = [f for f in model.F if model.origin_f[f] == k]

    # creamos la parte izquierda de la restricción
    lhs = sum(model.w[f,k,t] - model.w[f,k,t-1] for f in flights_from_k)
    return lhs <= model.D_k[k,t] # aplicamos la restricción
# con esta información creamos la restricción
model.DepartureCapacity = Constraint(model.K, model.T, rule=departure_capacity_constraint)
model.DepartureCapacity.pprint()


DepartureCapacity : Size=160, Index=K*T, Active=True
    Key            : Lower : Body                                                                                                            : Upper : Active
     ('STAR 1', 1) :  -Inf :             w[f1,STAR 1,1] - w[f1,STAR 1,0] + w[f2,STAR 1,1] - w[f2,STAR 1,0] + w[f3,STAR 1,1] - w[f3,STAR 1,0] :   1.0 :   True
     ('STAR 1', 2) :  -Inf :             w[f1,STAR 1,2] - w[f1,STAR 1,1] + w[f2,STAR 1,2] - w[f2,STAR 1,1] + w[f3,STAR 1,2] - w[f3,STAR 1,1] :   1.0 :   True
     ('STAR 1', 3) :  -Inf :             w[f1,STAR 1,3] - w[f1,STAR 1,2] + w[f2,STAR 1,3] - w[f2,STAR 1,2] + w[f3,STAR 1,3] - w[f3,STAR 1,2] :   1.0 :   True
     ('STAR 1', 4) :  -Inf :             w[f1,STAR 1,4] - w[f1,STAR 1,3] + w[f2,STAR 1,4] - w[f2,STAR 1,3] + w[f3,STAR 1,4] - w[f3,STAR 1,3] :   1.0 :   True
     ('STAR 1', 5) :  -Inf :             w[f1,STAR 1,5] - w[f1,STAR 1,4] + w[f2,STAR 1,5] - w[f2,STAR 1,4] + w[f3,STAR 1,5] - w[f3,STAR 1,4] :   1.0 :   True

In [61]:
################### Define Arrival Capacity constraint ####################
# simetrico al departure constrain
def arrival_capacity_constraint(model, k, t):
    if t == 0:
        return Constraint.Skip

    has_flights_to_k = [f for f in model.F if model.destination_f[f] == k]  
    if not has_flights_to_k:
        return Constraint.Skip
    
    flights_to_k = [f for f in model.F if model.destination_f[f] == k]

    lhs = sum(model.w[f,k,t] - model.w[f,k,t-1] for f in flights_to_k)
    return lhs <= model.A_k[k, t]

model.ArrivalCapacity = Constraint(model.K, model.T, rule=arrival_capacity_constraint)
model.ArrivalCapacity.pprint()


ArrivalCapacity : Size=40, Index=K*T, Active=True
    Key         : Lower : Body                                                                                                                                                                                                                                                                                                                                                                        : Upper : Active
     ('FAP', 1) :  -Inf :                         w[f1,FAP,1] - w[f1,FAP,0] + w[f2,FAP,1] - w[f2,FAP,0] + w[f3,FAP,1] - w[f3,FAP,0] + w[f4,FAP,1] - w[f4,FAP,0] + w[f5,FAP,1] - w[f5,FAP,0] + w[f6,FAP,1] - w[f6,FAP,0] + w[f7,FAP,1] - w[f7,FAP,0] + w[f8,FAP,1] - w[f8,FAP,0] + w[f9,FAP,1] - w[f9,FAP,0] + w[f10,FAP,1] - w[f10,FAP,0] + w[f11,FAP,1] - w[f11,FAP,0] + w[f12,FAP,1] - w[f12,FAP,0] :   1.0 :   True
     ('FAP', 2) :  -Inf :                         w[f1,FAP,2] - w[f1,FAP,1] + w[f2,FAP,2] - w[f2,FAP,1] + w[f3,FAP,2] - w[f3,FAP,1] + w[

# CONNVECTIVITY CONTRAINS

In [62]:
######## Connectivity constratins 1 ########
# Create a set that combines the indices you need for the constraint
model.T_fn_combined = Set(dimen=3, initialize=lambda model: ((f, n, t) for f in model.F for n in model.N_f[f] for t in model.T_fn[f,n]))
display(model.T_fn_combined) 

def flight_structure_constraints_1_rule(model, f, j, t):
    if t==0:
        return Constraint.Skip
    
    n = list(model.N_f[f]).index(j)

    if n < len(model.N_f[f]) - 1:
        m = n+1
        i = list(model.N_f[f])[m]

        arc = f"({j},{i})"
        l = model.l_fnm[f, arc]      
            
        return model.w[f, i, int(t + l)] - model.w[f, j, int(t)] <= 0
    else:
        return Constraint.Skip


model.flight_structure_constraints_1 = Constraint(model.T_fn_combined,rule=flight_structure_constraints_1_rule)

model.flight_structure_constraints_1.pprint()
    # It may be interesting to access to the index (n) of a given element j) 
    #n = N_f[f].index(j)
    # It may be also interesting to access to the element (m) of a fiven index (n + 1)
    #m = N_f[f][n+1]
    
#model.flight_structure_constraints_1 = Constraint(model.T_fn_combined, rule=flight_structure_constraints_1_rule)


T_fn_combined : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     3 :    Any :  360 : {('f1', 'STAR 1', np.float64(1.0)), ('f1', 'STAR 1', np.float64(2.0)), ('f1', 'STAR 1', np.float64(3.0)), ('f1', 'STAR 1', np.float64(4.0)), ('f1', 'STAR 1', np.float64(5.0)), ('f1', 'IAF 1', np.float64(5.0)), ('f1', 'IAF 1', np.float64(6.0)), ('f1', 'IAF 1', np.float64(7.0)), ('f1', 'IAF 1', np.float64(8.0)), ('f1', 'IAF 1', np.float64(9.0)), ('f1', 'HOLD 1', np.float64(5.0)), ('f1', 'HOLD 1', np.float64(6.0)), ('f1', 'HOLD 1', np.float64(7.0)), ('f1', 'HOLD 1', np.float64(8.0)), ('f1', 'HOLD 1', np.float64(9.0)), ('f1', 'HOLD 1', np.float64(10.0)), ('f1', 'HOLD 1', np.float64(11.0)), ('f1', 'HOLD 1', np.float64(12.0)), ('f1', 'HOLD 1', np.float64(13.0)), ('f1', 'HOLD 1', np.float64(14.0)), ('f1', 'FAP', np.float64(10.0)), ('f1', 'FAP', np.float64(11.0)), ('f1', 'FAP', np.float64(12.0)), ('f1', 'FAP', np.float64(13.0)), ('f1', 'FAP', np.float64(14.0)), ('

You should get this print:

T_fn_combined : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     3 :    Any :   78 : {('f1', 'A', 1.0), ('f1', 'A', 2.0), ('f1', 'A', 3.0), ('f1', 'q(A)', 1.0), ('f1', 'q(A)', 2.0), ('f1', 'q(A)', 3.0), ('f1', 'a', 6.0), ('f1', 'a', 7.0), ('f1', 'a', 8.0), ('f1', 'a', 9.0), ('f1', 'a', 10.0), ('f1', 'a', 11.0), ('f1', 'a', 12.0), ('f1', 'a', 13.0), ('f1', 'b', 14.0), ('f1', 'b', 15.0), ('f1', 'b', 16.0), ('f1', 'b', 17.0), ('f1', 'b', 18.0), ('f1', 'b', 19.0), ('f1', 'b', 20.0), ('f1', 'b', 21.0), ('f1', 'C', 15.0), ('f1', 'C', 16.0), ('f1', 'C', 17.0), ('f1', 'C', 18.0), ('f1', 'C', 19.0), ('f1', 'C', 20.0), ('f1', 'C', 21.0), ('f1', 'C', 22.0), ('f2', 'A', 1.0), ('f2', 'A', 2.0), ('f2', 'A', 3.0), ('f2', 'A', 4.0), ('f2', 'q(A)', 1.0), ('f2', 'q(A)', 2.0), ('f2', 'q(A)', 3.0), ('f2', 'q(A)', 4.0), ('f2', 'e', 6.0), ('f2', 'e', 7.0), ('f2', 'e', 8.0), ('f2', 'e', 9.0), ('f2', 'e', 10.0), ('f2', 'e', 11.0), ('f2', 'e', 12.0), ('f2', 'e', 13.0), ('f2', 'B', 13.0), ('f2', 'B', 14.0), ('f2', 'B', 15.0), ('f2', 'B', 16.0), ('f2', 'B', 17.0), ('f2', 'B', 18.0), ('f2', 'B', 19.0), ('f2', 'B', 20.0), ('f3', 'B', 17.0), ('f3', 'B', 18.0), ('f3', 'B', 19.0), ('f3', 'B', 20.0), ('f3', 'q(B)', 17.0), ('f3', 'q(B)', 18.0), ('f3', 'q(B)', 19.0), ('f3', 'q(B)', 20.0), ('f3', 'f', 20.0), ('f3', 'f', 21.0), ('f3', 'f', 22.0), ('f3', 'f', 23.0), ('f3', 'f', 24.0), ('f3', 'f', 25.0), ('f3', 'f', 26.0), ('f3', 'f', 27.0), ('f3', 'C', 23.0), ('f3', 'C', 24.0), ('f3', 'C', 25.0), ('f3', 'C', 26.0), ('f3', 'C', 27.0), ('f3', 'C', 28.0), ('f3', 'C', 29.0), ('f3', 'C', 30.0)}
0 A q(A) (A,q(A))
0 A q(A) (A,q(A))
0 A q(A) (A,q(A))
1 q(A) a (q(A),a)
1 q(A) a (q(A),a)
1 q(A) a (q(A),a)
2 a b (a,b)
2 a b (a,b)
2 a b (a,b)
2 a b (a,b)
2 a b (a,b)
2 a b (a,b)
2 a b (a,b)
2 a b (a,b)
3 b C (b,C)
3 b C (b,C)
3 b C (b,C)
3 b C (b,C)
3 b C (b,C)
3 b C (b,C)
3 b C (b,C)
3 b C (b,C)
0 A q(A) (A,q(A))
0 A q(A) (A,q(A))
0 A q(A) (A,q(A))
0 A q(A) (A,q(A))
1 q(A) e (q(A),e)
1 q(A) e (q(A),e)
1 q(A) e (q(A),e)
1 q(A) e (q(A),e)
2 e B (e,B)
2 e B (e,B)
2 e B (e,B)
2 e B (e,B)
2 e B (e,B)
2 e B (e,B)
2 e B (e,B)
2 e B (e,B)
0 B q(B) (B,q(B))
0 B q(B) (B,q(B))
0 B q(B) (B,q(B))
0 B q(B) (B,q(B))
1 q(B) f (q(B),f)
1 q(B) f (q(B),f)
1 q(B) f (q(B),f)
1 q(B) f (q(B),f)
2 f C (f,C)
2 f C (f,C)
2 f C (f,C)
2 f C (f,C)
2 f C (f,C)
2 f C (f,C)
2 f C (f,C)
2 f C (f,C)

In [63]:
######## Connectivity constratins 2 ########

def flight_structure_constraints_2_rule(model, f, k, t):
    # Solo tiempos del nodo de ORIGEN
    if k != model.origin_f[f]:
        return Constraint.Skip

    # Δt = (a_f - d_f) + A_f
    extra_t = (model.a_f[f] - model.d_f[f]) + model.A[f]
    tt = int(t) + int(extra_t)

    # Evitar pedir t fuera del horizonte
    if tt > model.T.last():
        return Constraint.Skip

    # w_{orig_f,t} - w_{dest_f, t + (a_f - d_f) + A_f} <= 0
    return model.w[f, model.origin_f[f], int(t)] - model.w[f, model.destination_f[f], tt] <= 0

model.flight_structure_constraints_2 = Constraint(model.T_fn_combined, rule=flight_structure_constraints_2_rule
)

model.flight_structure_constraints_2.pprint()


flight_structure_constraints_2 : Size=60, Index=T_fn_combined, Active=True
    Key                    : Lower : Body                            : Upper : Active
     ('f1', 'STAR 1', 1.0) :  -Inf :   w[f1,STAR 1,1] - w[f1,FAP,15] :   0.0 :   True
     ('f1', 'STAR 1', 2.0) :  -Inf :   w[f1,STAR 1,2] - w[f1,FAP,16] :   0.0 :   True
     ('f1', 'STAR 1', 3.0) :  -Inf :   w[f1,STAR 1,3] - w[f1,FAP,17] :   0.0 :   True
     ('f1', 'STAR 1', 4.0) :  -Inf :   w[f1,STAR 1,4] - w[f1,FAP,18] :   0.0 :   True
     ('f1', 'STAR 1', 5.0) :  -Inf :   w[f1,STAR 1,5] - w[f1,FAP,19] :   0.0 :   True
    ('f10', 'STAR 4', 1.0) :  -Inf : w[f10,STAR 4,1] - w[f10,FAP,15] :   0.0 :   True
    ('f10', 'STAR 4', 2.0) :  -Inf : w[f10,STAR 4,2] - w[f10,FAP,16] :   0.0 :   True
    ('f10', 'STAR 4', 3.0) :  -Inf : w[f10,STAR 4,3] - w[f10,FAP,17] :   0.0 :   True
    ('f10', 'STAR 4', 4.0) :  -Inf : w[f10,STAR 4,4] - w[f10,FAP,18] :   0.0 :   True
    ('f10', 'STAR 4', 5.0) :  -Inf : w[f10,STAR 4,5] - w[f10,FAP,

In [64]:
######## Connectivity constratins 3 ########
# def flight_structure_constraints_3_rule(model, f, _, t):
#     if f not in model.C:  # de aquí considerará solo f2 y f3
#         return Constraint.Skip

#     #  consideración origen f_= destino f
#     flights_with_cont = [f_ for f_ in model.F 
#                          if model.origin_f[f_] == model.destination_f[f]]
    
#     # If there are not continued flights skip it
#     if not flights_with_cont:
#         return Constraint.Skip
    
#     # The flight that is continuing (2nd)
#     f_ = flights_with_cont[0]  
    
#     t_dest = int(t - model.s_f[f])
    
#     if t_dest not in model.T:
#         return Constraint.Skip
    
#     return model.w[f_, model.origin_f[f_], t] - model.w[f, model.destination_f[f], t_dest] <= 0
# # Step 4. Add to model
# model.flight_structure_constraints_3 = Constraint(model.T_fn_combined, rule=flight_structure_constraints_3_rule)
# # Optional check
# model.flight_structure_constraints_3.pprint()

In [65]:
######## Connectivity constratins 4 ########
# It may be interesting to access to the first and last elements of a set 
#model.T_fn[f, j].first()
#model.T_fn[f, j].last()

# Note, please, that you need to do it not only for T_fn, yet for T. 
def connectivity_constraint_4_rule(model, f, n, t):
    
    # Skip the first time period, since t-1 doesn’t exist
    if t == model.T.first():
        return Constraint.Skip
    
    # Enforce monotonicity: once a flight reaches a node, it stays reached
    return model.w[f, n, t-1] - model.w[f, n, t] <= 0


# The constraint applies over flights f, nodes n, and time t
model.Connectivity4 = Constraint(
    [(f, n, t) for f in model.F for n in model.N_f[f] for t in model.T],
    rule=connectivity_constraint_4_rule
)

model.Connectivity4.pprint()

Connectivity4 : Size=1920, Index={('f1', 'STAR 1', 0), ('f1', 'STAR 1', 1), ('f1', 'STAR 1', 2), ('f1', 'STAR 1', 3), ('f1', 'STAR 1', 4), ('f1', 'STAR 1', 5), ('f1', 'STAR 1', 6), ('f1', 'STAR 1', 7), ('f1', 'STAR 1', 8), ('f1', 'STAR 1', 9), ('f1', 'STAR 1', 10), ('f1', 'STAR 1', 11), ('f1', 'STAR 1', 12), ('f1', 'STAR 1', 13), ('f1', 'STAR 1', 14), ('f1', 'STAR 1', 15), ('f1', 'STAR 1', 16), ('f1', 'STAR 1', 17), ('f1', 'STAR 1', 18), ('f1', 'STAR 1', 19), ('f1', 'STAR 1', 20), ('f1', 'STAR 1', 21), ('f1', 'STAR 1', 22), ('f1', 'STAR 1', 23), ('f1', 'STAR 1', 24), ('f1', 'STAR 1', 25), ('f1', 'STAR 1', 26), ('f1', 'STAR 1', 27), ('f1', 'STAR 1', 28), ('f1', 'STAR 1', 29), ('f1', 'STAR 1', 30), ('f1', 'STAR 1', 31), ('f1', 'STAR 1', 32), ('f1', 'STAR 1', 33), ('f1', 'STAR 1', 34), ('f1', 'STAR 1', 35), ('f1', 'STAR 1', 36), ('f1', 'STAR 1', 37), ('f1', 'STAR 1', 38), ('f1', 'STAR 1', 39), ('f1', 'STAR 1', 40), ('f1', 'IAF 1', 0), ('f1', 'IAF 1', 1), ('f1', 'IAF 1', 2), ('f1', 'IAF 1'

In [66]:
######## Connectivity constratins 5 ########

# Create a set that combines the indices you need for the constraint
model.N_f_combined = Set(dimen=2, initialize=lambda model: ((f, n) for f in model.F for n in model.N_f[f]))
display(model.N_f_combined) 

def connectivity_constraint_8_rule(model, f, n):

    # Access the last feasible time for this (f, n), (we can as T_fn is ordered)
    t_last = model.T_fn[f, n].last()

    # Enforce the condition
    return model.w[f, n, t_last] == 1


model.Connectivity8 = Constraint(model.N_f_combined, rule=connectivity_constraint_8_rule)

model.Connectivity8.pprint()

N_f_combined : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     2 :    Any :   48 : {('f1', 'STAR 1'), ('f1', 'IAF 1'), ('f1', 'HOLD 1'), ('f1', 'FAP'), ('f2', 'STAR 1'), ('f2', 'IAF 1'), ('f2', 'HOLD 1'), ('f2', 'FAP'), ('f3', 'STAR 1'), ('f3', 'IAF 1'), ('f3', 'HOLD 1'), ('f3', 'FAP'), ('f4', 'STAR 2'), ('f4', 'IAF 2'), ('f4', 'HOLD 2'), ('f4', 'FAP'), ('f5', 'STAR 2'), ('f5', 'IAF 2'), ('f5', 'HOLD 2'), ('f5', 'FAP'), ('f6', 'STAR 2'), ('f6', 'IAF 2'), ('f6', 'HOLD 2'), ('f6', 'FAP'), ('f7', 'STAR 3'), ('f7', 'IAF 3'), ('f7', 'HOLD 3'), ('f7', 'FAP'), ('f8', 'STAR 3'), ('f8', 'IAF 3'), ('f8', 'HOLD 3'), ('f8', 'FAP'), ('f9', 'STAR 3'), ('f9', 'IAF 3'), ('f9', 'HOLD 3'), ('f9', 'FAP'), ('f10', 'STAR 4'), ('f10', 'IAF 4'), ('f10', 'HOLD 4'), ('f10', 'FAP'), ('f11', 'STAR 4'), ('f11', 'IAF 4'), ('f11', 'HOLD 4'), ('f11', 'FAP'), ('f12', 'STAR 4'), ('f12', 'IAF 4'), ('f12', 'HOLD 4'), ('f12', 'FAP')}
Connectivity8 : Size=48, Index=N_f_combi

In [67]:
# Additional constraint to enforce W = 0 for all t < T_fn[f, j].first()

# Create a set that combines the indices you need for the constraint
model.N_f_combined = Set(dimen=2, initialize=lambda model: ((f, n) for f in model.F for n in model.N_f[f]))
display(model.N_f_combined) 

def connectivity_constraint8(model, f, j, t):
    if t <= model.T_fn[f, j].first() - 1:
        return model.w[f, j, t]==0
    else:
        return Constraint.Skip
model.connectivity_constraint8 = Constraint(model.N_f_combined, model.T, rule=connectivity_constraint8)
model.connectivity_constraint8.pprint()

(type=<class 'pyomo.core.base.set.OrderedScalarSet'>) on block AMAN with a new
Component (type=<class 'pyomo.core.base.set.AbstractOrderedScalarSet'>). This
is usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
N_f_combined : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     2 :    Any :   48 : {('f1', 'STAR 1'), ('f1', 'IAF 1'), ('f1', 'HOLD 1'), ('f1', 'FAP'), ('f2', 'STAR 1'), ('f2', 'IAF 1'), ('f2', 'HOLD 1'), ('f2', 'FAP'), ('f3', 'STAR 1'), ('f3', 'IAF 1'), ('f3', 'HOLD 1'), ('f3', 'FAP'), ('f4', 'STAR 2'), ('f4', 'IAF 2'), ('f4', 'HOLD 2'), ('f4', 'FAP'), ('f5', 'STAR 2'), ('f5', 'IAF 2'), ('f5', 'HOLD 2'), ('f5', 'FAP'), ('f6', 'STAR 2'), ('f6', 'IAF 2'), ('f6', 'HOLD 2'), ('f6', 'FAP'), ('f7', 'STAR 3'), ('f7', 'IAF 3'), ('f7', 'HOLD 3'), ('f7', 'FAP'), ('f8', 'STAR 3'), ('f8', 'IAF 3'), ('f8', 'HOLD 3'), ('f8', 'FAP'), ('f9', 'STAR 3'), ('f9', 'IAF 3'), ('f9', 'HOLD

# SOLVER

In [68]:
# Solver call
solver = SolverFactory('glpk')
#solver = SolverFactory('cplex_direct')
result = solver.solve(model)
model.display()
pyomo.environ.value(model.OBJ)
objective_val = model.OBJ()
print("Objective value after minimizing:",objective_val)

Model AMAN

  Variables:
    w : Size=1968, Index={('f1', 'STAR 1'), ('f1', 'IAF 1'), ('f1', 'HOLD 1'), ('f1', 'FAP'), ('f2', 'STAR 1'), ('f2', 'IAF 1'), ('f2', 'HOLD 1'), ('f2', 'FAP'), ('f3', 'STAR 1'), ('f3', 'IAF 1'), ('f3', 'HOLD 1'), ('f3', 'FAP'), ('f4', 'STAR 2'), ('f4', 'IAF 2'), ('f4', 'HOLD 2'), ('f4', 'FAP'), ('f5', 'STAR 2'), ('f5', 'IAF 2'), ('f5', 'HOLD 2'), ('f5', 'FAP'), ('f6', 'STAR 2'), ('f6', 'IAF 2'), ('f6', 'HOLD 2'), ('f6', 'FAP'), ('f7', 'STAR 3'), ('f7', 'IAF 3'), ('f7', 'HOLD 3'), ('f7', 'FAP'), ('f8', 'STAR 3'), ('f8', 'IAF 3'), ('f8', 'HOLD 3'), ('f8', 'FAP'), ('f9', 'STAR 3'), ('f9', 'IAF 3'), ('f9', 'HOLD 3'), ('f9', 'FAP'), ('f10', 'STAR 4'), ('f10', 'IAF 4'), ('f10', 'HOLD 4'), ('f10', 'FAP'), ('f11', 'STAR 4'), ('f11', 'IAF 4'), ('f11', 'HOLD 4'), ('f11', 'FAP'), ('f12', 'STAR 4'), ('f12', 'IAF 4'), ('f12', 'HOLD 4'), ('f12', 'FAP')}*T
        Key                   : Lower : Value : Upper : Fixed : Stale : Domain
             ('f1', 'FAP', 0) :     0 : 

In [69]:
# Prints: one can print the complete model or just some elements of it

# model.pprint()
#model.obj.pprint()
#model.departure_capacity_constraint.pprint()
#model.arrival_capacity_constraint.pprint()
#model.sector_capacity_constraint.pprint()
#model.flight_structure_constraints_1.pprint()
#model.Connectivity2.pprint()
#model.Connectivity3_bis.pprint()
#model.flight_structure_constraints_4.pprint()
#model.connectivity_constraint4_bis.pprint()
#model.flight_structure_constraints_5.pprint()
# model.w.pprint()
model.

SyntaxError: invalid syntax (429902727.py, line 15)

In [ ]:
#plots and Tables
